<a href="https://colab.research.google.com/github/JasmineVan/Spark_Aggregate_By_Concaternating/blob/main/Spark_Agg_Concaternating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [139]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [140]:
import findspark
findspark.init()

In [141]:
from pyspark import SparkContext

sc = SparkContext("local", "cau 1")

ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [143]:
from pyspark.sql import SQLContext
import pandas as pd

sqlc = SQLContext(sc)

In [144]:
# Please change file dir
path = '/content/drive/MyDrive/Colab_Notebooks/DLL/data.csv'

In [145]:
df = sqlc.read.csv(path, sep=';', header=True)
df.show()

+------+--------------------+--------+----------------+-----+----------+--------------+
|BillNo|            Itemname|Quantity|            Date|Price|CustomerID|       Country|
+------+--------------------+--------+----------------+-----+----------+--------------+
|536365|WHITE HANGING HEA...|       6|01.12.2010 08:26| 2,55|     17850|United Kingdom|
|536365| WHITE METAL LANTERN|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|CREAM CUPID HEART...|       8|01.12.2010 08:26| 2,75|     17850|United Kingdom|
|536365|KNITTED UNION FLA...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|RED WOOLLY HOTTIE...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|SET 7 BABUSHKA NE...|       2|01.12.2010 08:26| 7,65|     17850|United Kingdom|
|536365|GLASS STAR FROSTE...|       6|01.12.2010 08:26| 4,25|     17850|United Kingdom|
|536366|HAND WARMER UNION...|       6|01.12.2010 08:28| 1,85|     17850|United Kingdom|
|536366|HAND WARMER RED P...|   

In [146]:
df.printSchema()

root
 |-- BillNo: string (nullable = true)
 |-- Itemname: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [147]:
from pyspark.sql.types import IntegerType
df = df.withColumn('BillNo', df['BillNo'].cast(IntegerType()))

In [148]:
df = df.drop("Quantity", "Date", "Price", "CustomerID", "Country") 

import pyspark.sql.functions as func
df = df.groupby("BillNo")\
.agg(func.concat_ws(", ", func.collect_list(df.Itemname)).alias("Itemname"))\

df.show()

+------+--------------------+
|BillNo|            Itemname|
+------+--------------------+
|536532|BOX OF 24 COCKTAI...|
|537632|          AMAZON FEE|
|538708|BOX OF VINTAGE JI...|
|538877|RED RETROSPOT BOW...|
|538993|ANTIQUE GLASS HEA...|
|539735|PORCELAIN BUTTERF...|
|539958|DENIM PATCH PURSE...|
|540562|MINI FUNKY DESIGN...|
|540563|VICTORIAN SEWING ...|
|541008|PAPER BUNTING WHI...|
|542058|ASSORTED LAQUERED...|
|542239|COOK WITH WINE ME...|
|542694|ASSORTED COLOURS ...|
|542723|SOMBRERO, DAISIES...|
|543751|WOODEN ROUNDERS G...|
|544198|VINTAGE UNION JAC...|
|544205|INFLATABLE POLITI...|
|544702|CHILDRENS APRON S...|
|545638|SET/3 RED GINGHAM...|
|545649|ZINC METAL HEART ...|
+------+--------------------+
only showing top 20 rows



In [149]:
df = df.drop("BillNo") 

In [152]:
# df.repartition(1).write.format("csv").save("/content/drive/MyDrive/Colab_Notebooks/DLL/baskets.csv")
df.coalesce(1)\
  .write.format("csv")\
  .mode("overwrite")\
  .save("/content/drive/MyDrive/Colab_Notebooks/DLL/baskets.csv", header = True)